In [ ]:
import numpy as np
import miditoolkit
import math
import os
from tqdm import tqdm

In [ ]:
# midi folder
# midi file -> quantize
# quantized midi file -> graph

In [136]:
midi_file = "/Users/kshitij1/Documents/GitHub/Clarinet/Data/Midi/Collection/Original Collection/001.mid"

In [147]:
def process_midiFile(midi_file):
    mid_in=miditoolkit.midi.parser.MidiFile(midi_file)    
    notes = []
    for i in range(len(mid_in.instruments)):
        notes.extend(mid_in.instruments[i].notes)
    notes.sort(key=lambda x: x.start)
    return notes

In [ ]:
# start = notes[0].start

# for note in notes:
#     note.start = note.start - start
#     note.end = note.end - start

# notes[:5]

In [148]:
notes = process_midiFile(midi_file)

In [149]:
def quantize(notes, planck=1, rest=True):
    start = notes[0].start

    for note in notes:
        note.start = note.start - start
        note.end = note.end - start
    
    notes.sort(key=lambda x: x.end)
    end = notes[-1].end
    notes.sort(key=lambda x: x.start)

    quantized_notes = []
    for i in range(int(end/planck)):
        quantized_notes.append([])
    
    for note in notes:
        cur_start = note.start
        cur_end = note.end
        first_idx = int(cur_start/planck)
        last_idx = int(cur_end/planck)
        pitch = note.pitch
        vel = note.velocity
        for i in range(first_idx, last_idx):
            quantized_notes[i].append((pitch,vel))
            
    if rest:
        for i in range(len(quantized_notes)):
            quantized_notes[i].append((-1,-1))
    else:
        for i in range(len(quantized_notes)):
            if len(quantized_notes[i]) == 0:
                quantized_notes[i].append((-1,-1))
    return quantized_notes

In [150]:
qn = quantize(notes, planck=1)

In [ ]:
def p(pitch1, pitch2):
    return 1

def q(pitch1, pitch2):
    return 1

def weight(note1,note2,type="prob"):
    p1 = note1[0]
    p2 = note2[0]
    if type == "prob":
        return -1 * math.log(p(p1,p2)/q(p1,p2))
    return 1

In [ ]:
def makeMelodicTransitionMat(midi_folder):
    mat = np.zeros((129,129))
    for f in tqdm(midi_folder):
        mid_in=miditoolkit.midi.parser.MidiFile(f)    
        notes = mid_in.instruments[0].notes
        qn = quantize(notes, planck=1, rest=False)
        for i in range(len(qn)-1):
            note1 = qn[i][0]
            note2 = qn[i+1][0]
            p1 = note1[0]
            p2 = note2[0]
            mat[p1][p2] += 1
    for i in range(129):
        if np.sum(mat[i]) != 0:
            mat[i] = mat[i]/np.sum(mat[i])
    return mat
    

In [ ]:
def makeNonMelodicTransitionMat(midi_folder):
    mat = np.zeros((129,129))
    for f in tqdm(midi_folder):
        mid_in=miditoolkit.midi.parser.MidiFile(f)    
        for channel in range(0,3):
            notes = mid_in.instruments[channel].notes
            qn = quantize(notes, planck=1)
            for i in range(len(qn)-1):
                box1 = qn[i]
                box2 = qn[i+1]
                for _ in range(5):
                    note1 = box1[np.random.randint(len(box1))]
                    note2 = box2[np.random.randint(len(box2))]
                    p1 = note1[0]
                    p2 = note2[0]
                    mat[p1][p2] += 1
    for i in range(129):
        if np.sum(mat[i]) != 0:
            mat[i] = mat[i]/np.sum(mat[i])
    return mat
    

In [ ]:
def dumpMelodic(midi_folder,output_folder,num_files=5):
    files = [midi_folder+"/"+f for f in os.listdir(midi_folder) if f.endswith(".mid")]
    if num_files>0:
        files = files[:num_files]
    mat = makeMelodicTransitionMat(files)
    np.save(output_folder+"/melodic.npy",mat)

In [ ]:
def dumpNonMelodic(midi_folder,output_folder,num_files=5):
    files = [midi_folder+"/"+f for f in os.listdir(midi_folder) if f.endswith(".mid")]
    files = files[:num_files]
    mat = makeNonMelodicTransitionMat(files)
    np.save(output_folder+"/nonmelodic.npy",mat)

In [ ]:
output_folder = "../../../Data/Numpy/noBERT"
midi_folder = "/Users/kshitij1/Documents/GitHub/Clarinet/Data/Midi/Collection/Original Collection"

In [ ]:
dumpMelodic(midi_folder,output_folder,-1)

In [ ]:
dumpNonMelodic(midi_folder,output_folder,5)

In [ ]:
def p(pitch1, pitch2):
    return 1

def q(pitch1, pitch2):
    return 1

def weight(note1,note2,type="prob"):
    p1 = note1[0]
    p2 = note2[0]
    if type == "prob":
        return -1 * math.log(p(p1,p2)/q(p1,p2))
    return 1

In [ ]:
def dumpWeight(numpy_folder):
    melodic = np.load(numpy_folder+"/melodic.npy")
    nonmelodic = np.load(numpy_folder+"/nonmelodic.npy")
    mat = np.zeros((129,129))
    for i in range(129):
        for j in range(129):
            if nonmelodic[i][j] != 0:
                mat[i][j] = melodic[i][j]/nonmelodic[i][j]
            else:
                mat[i][j] = melodic[i][j] * pow(10,5)
            if mat[i][j] == 0:
                mat[i][j] = pow(10,-5)
            mat[i][j] = -1 * math.log(mat[i][j])
    np.save(numpy_folder+"/weight.npy",mat)

In [ ]:
dumpWeight(output_folder)

In [ ]:
def loadWeight(numpy_folder):
    return np.load(numpy_folder+"/weight.npy")

In [ ]:
weights = loadWeight(output_folder)

In [ ]:
'''
i -> (i+2), (i+4), (i+5), (i+7), (i+9), (i+11)
7 > 5 > 4 > 2~9 > 11
'''
def check(weights):
    i = 60
    for j in range(1,12):
        print(i,i+j, weights[i][i+j])
check(weights)

In [152]:
class Graph:
    def __init__(self, midi_file,weight_matrix):
        self.midi_file = midi_file
        self.box_list = quantize(process_midiFile(midi_file),planck=1)
        self.weight_matrix = weight_matrix
        self.layers = [[] for i in range(len(self.box_list))]
        # each layer will contain [ [(pitch,velocity,[list of weights corresponding next layer])], ... ]
    
    def create_graph(self):
        for i in range(len(self.box_list)-1):
            box1 = self.box_list[i]
            box2 = self.box_list[i+1]
            for note1 in box1:
                weight_list = []
                for note2 in box2:
                    weight_list.append(self.get_weights(note1,note2))
                self.layers[i].append([note1, weight_list])
    
    def get_weights(self,note1,note2):
        p1 = note1[0]
        p2 = note2[0]
        return self.weight_matrix[p1][p2]
            

In [154]:
G = Graph(midi_file,weights)
G.create_graph()